In [39]:
import numpy as np
import pandas as pd
import math
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, Ridge, RidgeCV, LassoCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, precision_recall_curve,precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Binarizer
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

In [40]:
#reading the data

dataset = pd.read_csv('caesarian.csv')
#print(dataset)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
Unnamed: 0        80 non-null int64
age               80 non-null object
delivery_no       80 non-null object
delivery_time     80 non-null int64
blood_pressure    80 non-null int64
heart_problem     80 non-null int64
caesarian         80 non-null int64
dtypes: int64(5), object(2)
memory usage: 4.5+ KB


In [41]:
#Imputating of missing values

dataset =dataset.replace('-','NaN')

#Replacing the missing age values with the integer equal to the round down of the mean of age

age=dataset['age']
imp_mean =SimpleImputer(missing_values=np.NaN, strategy='mean' )
fit_age=imp_mean.fit(age.values.reshape(-1, 1))
imp_age=imp_mean.transform(age.values.reshape(-1, 1))
imp_age=np.floor(imp_age)
#print(imp_age.shape)
#print(imp_age)


#Replacing the missing delivery number values with the most frequent delivery number

delivery_no =dataset['delivery_no']
imp_most_freq =SimpleImputer(missing_values='NaN',strategy='most_frequent')
fit_delivery_no=imp_most_freq.fit(delivery_no.values.reshape(-1, 1))
imp_delivery_no=imp_most_freq.transform(delivery_no.values.reshape(-1, 1))

#print(imp_delivery_no.shape)
#print(imp_delivery_no)

In [42]:
#Encoding the categorical variables using the one-hot encoding


inp1=pd.DataFrame(imp_age)
inp2=pd.DataFrame(imp_delivery_no)
#inp2=pd.get_dummies(pd.DataFrame(imp_delivery_no))
inp3=pd.get_dummies(dataset['delivery_time'])
inp4=pd.get_dummies(dataset['blood_pressure'])
inp5=dataset['heart_problem']

frames1=[inp1,inp2,inp3,inp4,inp5]
inp=pd.concat(frames1,axis=1)
out = dataset['caesarian']
#print(inp.shape)
#print(out.shape)


In [43]:
# scaling the  data

scaler = preprocessing.StandardScaler().fit(inp)
scaled = scaler.transform(inp)
print(scaled.shape)
#print(scaled)

(80, 9)


C:\Users\pantp\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pantp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [44]:
# train logistic regression & cross validation using accuracy scoring
#default scoring option used is accuracy

clf_L2 = LogisticRegressionCV(Cs=100, cv=3, penalty='l2', random_state=0, multi_class='multinomial',
                              solver='lbfgs', n_jobs=-1,max_iter=1000)
clf_L2.fit (scaled, out)

print('1/alpha = ', clf_L2.C_) 
print('coef = ', clf_L2.coef_)
print('accuracy = ', clf_L2.score(scaled,out))


#Predicting the output data:

pred = clf_L2.predict(scaled)
print(pred)
#print('accuracy = ', clf_L2.score(scaled,pred))

#Average predicted accuracy

avg=cross_val_score(clf_L2, scaled, out, groups=None, scoring='accuracy',cv=3)
print(avg)
print('Average predicted accuracy =',avg.mean())


1/alpha =  [0.06734151]
coef =  [[ 0.0330783   0.06878643  0.12774803 -0.05571059 -0.09866472  0.13769898
  -0.17581956  0.06531996  0.22321539]]
accuracy =  0.7125
[1 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0
 0 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1
 1 0 1 1 1 0]
[0.67857143 0.69230769 0.57692308]
Average predicted accuracy = 0.6492673992673993


In [45]:
#check the precision, recall and accuracy

#print(accuracy_score(out,pred))
print(precision_recall_curve(out,pred))
print(precision_score(out,pred))
print(recall_score(out,pred))
print(pred)
p_pred=clf_L2.predict_proba(scaled)
#print(p_pred)


#probabilities:

#print(p_tp)
#print(p_tn)
#print(p_fp)
#print(p_fn)

#confusion matrix and expected value

#C = confusion_matrix(out,pred)
#print('Confusion matrix = ', C)
#p_tn, p_fp, p_fn, p_tp = C.ravel()/np.sum(C)
#EV = 200*p_tp-400*p_tn-450*p_fn +0*p_fp
#print('Expected_value', EV)


(array([0.575     , 0.73469388, 1.        ]), array([1.       , 0.7826087, 0.       ]), array([0, 1], dtype=int64))
0.7346938775510204
0.782608695652174
[1 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0
 0 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1
 1 0 1 1 1 0]


In [46]:
#create a function called underlying_EV_score(out,p_pred) that calculates the EV of an estimator

def underlying_EV_score(out,p_pred,sample_weight=None):
    pred = np.array([p_pred >= 0.66], dtype=np.int32)[0]
    C = confusion_matrix(out,pred)
    print('Confusion matrix = ', C)
    p_tn, p_fp, p_fn, p_tp = C.ravel()/np.sum(C)
    EV = 200*p_tp-400*p_tn-450*p_fn +0*p_fp
    print('Expected_value', EV)
    return EV

print('expected_value',EV)
print('Confusion matrix = ', C)


expected_value -71.25
Confusion matrix =  [[21 13]
 [10 36]]


In [47]:
#making a scorer from performance matrix

myEVscorer = make_scorer(underlying_EV_score, greater_is_better=True, needs_proba=True, needs_threshold=False)
print(myEVscorer)

# train logistic regression & cross validation using expected value as scoring function

ev_clf_L2 = LogisticRegressionCV(Cs=100,cv=3,penalty='l2',random_state=0,multi_class='multinomial',solver='lbfgs',
                                 max_iter=1000, n_jobs=-1, scoring=myEVscorer)
ev_clf_L2.fit(scaled,out)
print(ev_clf_L2)

#predicting the data

pred_2=ev_clf_L2.predict(scaled)
print(pred_2)
p_preda=ev_clf_L2.predict_proba(scaled)
#print(p_preda)


make_scorer(underlying_EV_score, needs_proba=True)
LogisticRegressionCV(Cs=100, class_weight=None, cv=3, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=1000,
           multi_class='multinomial', n_jobs=-1, penalty='l2',
           random_state=0, refit=True,
           scoring=make_scorer(underlying_EV_score, needs_proba=True),
           solver='lbfgs', tol=0.0001, verbose=0)
[1 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0
 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1
 1 0 1 1 0 0]


In [48]:
#coefficient values

print('1/alpha = ', ev_clf_L2.C_) 
print('coef = ', ev_clf_L2.coef_)

#accuracy of the model
print('Accuracy',accuracy_score(out,pred_2))

#expecte value on whole training data:
C_new = confusion_matrix(out,pred_2)
print('Confusion matrix = ', C_new)
p_tn, p_fp, p_fn, p_tp = C_new.ravel()/np.sum(C_new)
EV_new = 200*p_tp-400*p_tn-450*p_fn +0*p_fp
print('EV Value for this model',EV_new)


# print('EV  scores = ', ev_clf_L2.scores_[1].mean(axis=0))
# print('C inv.reg  = ', ev_clf_L2.Cs_)

mean_EV = ev_clf_L2.scores_[1].mean(axis=0)
se_EV = ev_clf_L2.scores_[1].std(axis=0)/np.sqrt(ev_clf_L2.cv)
print('Mean EV = ', mean_EV[ev_clf_L2.Cs_ == ev_clf_L2.C_])
print('SE EV = ', se_EV[ev_clf_L2.Cs_ == ev_clf_L2.C_])


1/alpha =  [0.20565123]
coef =  [[ 0.03456609  0.08717588  0.16484285 -0.06559926 -0.13360276  0.17618054
  -0.21966848  0.07747077  0.28217377]]
Accuracy 0.725
Confusion matrix =  [[24 10]
 [12 34]]
EV Value for this model -102.5
Mean EV =  [-194.18498168]
SE EV =  [19.89544824]


In [49]:
#predicting the final question_9:

#reading the data
dataset_final = pd.read_csv('caesarian_final_pred.csv')
#print(dataset_final)

#Encoding the categorical variables using the one-hot encoding

inp1_final=dataset_final['age']
inp2_final=dataset_final['delivery_no']
#inp2_final=pd.get_dummies(pd.DataFrame(imp_delivery_no))
inp3_final=dataset_final['delivery_time']
inp4_final=dataset_final['blood_pressure']
inp5_final=dataset_final['heart_problem']

frames1_final=[inp1_final,inp2_final,inp3_final,inp4_final,inp5_final]
inp_final=pd.concat(frames1_final,axis=1)
print(inp_final)

out_final = dataset_final['caesarian']

   age  delivery_no  delivery_time  blood_pressure  heart_problem
0   35            1              2               0              0


In [50]:
#making the dimensions same:

new_1=np.append(inp_final,0).reshape(1, -1)
new_2=np.append(new_1,0).reshape(1, -1)
new_3=np.append(new_2,0).reshape(1, -1)
new_4=np.append(new_3,0).reshape(1, -1)
#print(new_4)
                
# scaling the  data

scaler_final = preprocessing.StandardScaler().fit(new_4)
scaled_final = scaler_final.transform(new_4)
#print(scaled_final.shape)

C:\Users\pantp\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\pantp\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [51]:
#predicting the output using 1st model:

output_1=clf_L2.predict(scaled_final)
print('class',output_1)

p_output_1=clf_L2.predict_proba(scaled_final)
print('class_probability',p_output_1)

#predicting the output using 2nd model:
output_2=ev_clf_L2.predict(scaled_final)
print('class',output_2)

p_output_2=ev_clf_L2.predict_proba(scaled_final)
print('class_probability',p_output_2)

class [1]
class_probability [[0.41175073 0.58824927]]
class [1]
class_probability [[0.40500112 0.59499888]]
